In [1]:
import torch 
from torch import nn 
import torch; torch.manual_seed(42)
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
import pandas as pd
from os import listdir, path
import glob
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.distributions as dist
import os 

In [3]:


data_directory = 'acted'  # Replace with the path to your data directory
output = 'out'
# Get a list of all files in the data directory
file_list = listdir(data_directory)

# Loop through each file and process it
for file_name in file_list:
    file_path = os.path.join(data_directory, file_name)

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path,skiprows=17,sep="  ",index_col=0,header=None,engine='python')

   
    # Save the modified DataFrame back to a new CSV file
    output_file_path = os.path.join(output, f"{file_name}")
    df.to_csv(output_file_path, index=False, header=False)

print("Processing completed.")


Processing completed.


In [8]:

tmps = listdir("acted") # current directory
ems = [] #TRUE emotions from file names
for i in tmps:
    ems.append(i.split("_")[-2])
em = np.unique(ems)
em

array(['angry', 'fear', 'happy', 'sad'], dtype='<U5')

In [4]:
tmps = listdir("out") # current directory
ems = []
for i in range(len(ems)):
    if ems[i]=='angry':
        ems[i]= 0
    elif ems[i]== 'sad':
        ems[i]=1
    elif ems[i]== 'happy':
        ems[i]=2
    elif ems[i]== 'fear':
        ems[i]=3
   

In [ ]:


# Define the path to the directory containing the mocap data files
data_dir = 'out'

# List all the mocap data files
data_files = glob.glob(data_dir + '/*.csm')

# Initialize empty lists to store the data
mocap_data = []
emotion_labels = []
length = []
# Load the data from each file
for file in data_files:
    # Load the file and extract the coordinates
    # file_data = np.genfromtxt(file, delimiter=',')
    file_data = pd.read_csv(file, sep=',')
    # print(file_data.shape)
    if file_data.shape[1] == 102:
        file_data = file_data.iloc[ : , :-6]
    

    
    min_val = np.min(file_data)
    normalized_data = file_data - min_val
    range_val = np.max(file_data) - min_val
    normalized_data /= range_val
    file_data = normalized_data
    
#     # Append the coordinates to the mocap_data list
    mocap_data.append(file_data)
    # emotion_labels.append(ems)


    


max_length = max([sequence.shape[0] for sequence in mocap_data])

padded_mocap_data = []
for sequence in mocap_data:
    print(sequence.iloc[-1,:])
    num_frames_to_pad = max_length - sequence.shape[0]
    last_row = sequence.iloc[-1, :]  # Get the last row
    padded_sequence = np.concatenate((sequence, np.tile(last_row, (num_frames_to_pad, 1))), axis=0)
    padded_mocap_data.append(padded_sequence)
# Convert the emotion labels to a NumPy array
emotion_label = np.array(ems)

# Perform any necessary preprocessing steps on the mocap data and emotion labels
# For example, you can normalize the data or encode the emotion labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_mocap_data, emotion_label, test_size=0.2, random_state=42)
print(max_length)

In [42]:
def kl_divergence_loss(mu, logvar):
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return kl_loss

def reconstruction_loss(input,target):
    recon_loss = nn.CrossEntropyLoss()(input,target)
    return recon_loss

def total_loss( mu, logvar,input,target):
    kl_loss = kl_divergence_loss(mu, logvar)
    recon_loss = reconstruction_loss(input,target)

    total_loss = 0.005 * kl_loss + 0.95* recon_loss
    return total_loss




In [43]:
class BatchFlatten(nn.Module):
    def __init__(self):
        super(BatchFlatten,self).__init__()
        self._name = 'batch_flatten'
    def forward(self, x):
        return x.view(x.shape[0],-1)

In [51]:
class VPoser(nn.Module):
    def __init__(self):
        super(VPoser, self).__init__()

        num_neurons, self.latentD = 512,512
        nb_frames = 4003

        self.num_joints = 32
        n_features = self.num_joints * 3 * nb_frames

        self.encoder_net = nn.Sequential(
            BatchFlatten(),
            nn.BatchNorm1d(n_features),
            nn.Linear(n_features, num_neurons),
            nn.LeakyReLU(),
            nn.BatchNorm1d(num_neurons),
            nn.Dropout(0.1),
            nn.Linear(num_neurons, num_neurons),
            nn.LeakyReLU(),
            nn.Linear(num_neurons, self.latentD),
            # NormalDistDecoder(num_neurons, self.latentD)
            
            
            
        )

        self.decoder_net = nn.Sequential(
            nn.Linear(self.latentD, num_neurons),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(num_neurons, num_neurons),
            nn.LeakyReLU(),
            nn.Linear(num_neurons, n_features),
            # ContinousRotReprDecoder(),
        )
        self.mu_net = nn.Linear(512, 512)
        self.logvar_net = nn.Linear(512,512)
        # self.decoder_net = NormalDistDecoder(self.latentD, 512)




    def encode(self, pose_body):
        '''
        :param Pin: Nx(numjoints*3)
        :param rep_type: 'matrot'/'aa' for matrix rotations or axis-angle
        :return:
        
        '''
        # print('posebody', pose_body.shape)
        # self.mu_net = nn.Linear(32, 512)
        # self.logvar_net = nn.Linear(32,512)
        q_z = self.encoder_net(pose_body)
        # print("qz",q_z.shape)
        mu = self.mu_net(q_z)
        logvar = self.logvar_net(q_z)
        return q_z, mu, logvar

    def decode(self, Zin):
        # print('zin' ,Zin.shape)
        bs = Zin.shape[0]
        # print(bs)

        prec = self.decoder_net(Zin)
        prec1 = prec.view(bs,-1,96)
        # print('prec' ,prec1.shape)

        return {
            # 'pose_body': matrot2aa(prec.view(-1, 3, 3)).view(bs, -1, 3),
            'pose_body_matrot': prec.view(bs,-1,96)
        }


    def forward(self, pose_body):    

        q_z, mu, logvar = self.encode(pose_body)
        # print(q_z.shape)
        # q_z_sample = q_z.rsample()
        q_z_sample = self.reparameterize(mu, logvar)
        # print('qz sample ', q_z_sample.shape)

        decode_results = self.decode(q_z_sample)
        pose_body_decoded = decode_results['pose_body_matrot']  # Extract the decoded pose tensor

        # decode_results.update({'poZ_body_mean': q_z.mean, 'poZ_body_std': q_z.scale, 'q_z': q_z})
        return pose_body_decoded, mu , logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    
    
    def sample_poses(self, num_poses, seed=None):
        np.random.seed(seed)

        some_weight = [a for a in self.parameters()][0]
        dtype = some_weight.dtype
        device = some_weight.device
        self.eval()
        with torch.no_grad():
            Zgen = torch.tensor(np.random.normal(0., 1., size=(num_poses, self.latentD)), dtype=dtype, device=device)

        return self.decode(Zgen)
    

In [52]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset

# Convert the mocap data and emotion labels to NumPy arrays
X_train_array = np.array(X_train, dtype=np.float32)
y_train_array = np.array(y_train, dtype=np.compat.long)
X_test_array = np.array(X_test, dtype=np.float32)
y_test_array = np.array(y_test, dtype=np.compat.long)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_array)
y_train_tensor = torch.tensor(y_train_array)
X_test_tensor = torch.tensor(X_test_array)
y_test_tensor = torch.tensor(y_test_array)

# Create a custom dataset class to load the data
class MocapDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create DataLoader for training and testing sets
train_dataset = MocapDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=False)

test_dataset = MocapDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [54]:


# Define the VPoser model
model = VPoser()
optimizer = optim.Adam(model.parameters(), lr=0.000001)
criterion = nn.CrossEntropyLoss()
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_labels in train_loader:
        # print(batch_data[0][0])
        optimizer.zero_grad()
        outputs, mu, logvar = model(batch_data)
        # print(outputs[0][0])
       
        
        labels = batch_labels.repeat_interleave(96)
        # print(labels.shape)

# Reshape the tensor to size (128, 3)
        labels = labels.reshape(-1,96)
# ENCODE THE LABELS AND DO A CLUSTERING  SEE THE DISTROBUTION OF LATENT 
        # print(labels.shape)
        labels = labels.long()    
        # loss = criterion(outputs, labels)
        loss = total_loss(mu,logvar,outputs,labels) 
        optimizer.step()



        mse_criterion = torch.nn.MSELoss()
        mse = mse_criterion(outputs, batch_data)
    print("Mean Squared Error:", mse.item())

      
       

        
# LATENT PROBABILITY  
    # Print the loss at the end of each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss}")


Mean Squared Error: 0.35365650057792664
Epoch 1/10, Loss: 8.478814125061035
Mean Squared Error: 0.3543563783168793
Epoch 2/10, Loss: 8.47760009765625
Mean Squared Error: 0.354004442691803
Epoch 3/10, Loss: 8.4784574508667
Mean Squared Error: 0.35358425974845886
Epoch 4/10, Loss: 8.486741065979004
Mean Squared Error: 0.3542534410953522
Epoch 5/10, Loss: 8.475272178649902
Mean Squared Error: 0.35412371158599854
Epoch 6/10, Loss: 8.491087913513184
Mean Squared Error: 0.35288870334625244
Epoch 7/10, Loss: 8.480051040649414
Mean Squared Error: 0.3540427088737488
Epoch 8/10, Loss: 8.473740577697754
Mean Squared Error: 0.3533070981502533
Epoch 9/10, Loss: 8.47286319732666
Mean Squared Error: 0.3539043664932251
Epoch 10/10, Loss: 8.45984935760498


In [57]:
model.eval()
test_loss = 0.0
num_samples = 0
for batch_data, batch_labels in test_loader:
    outputs, mu, logvar = model(batch_data)
    labels = batch_labels.repeat_interleave(96)
    labels = labels.reshape(-1,96)
    labels = labels.long()    
    # loss = criterion(outputs,labels)
    loss = total_loss(mu,logvar,outputs,labels)
    mse_criterion = torch.nn.MSELoss()
    mse = mse_criterion(outputs, batch_data)
print("Mean Squared Error:", mse.item())
    
print(f" Loss: {loss} ")




    


Mean Squared Error: 0.37369003891944885
 Loss: 8.346708297729492 
